In [1]:
#install wandb
!pip install -q wandb --upgrade
import wandb

     |████████████████████████████████| 1.8 MB 5.4 MB/s 
     |████████████████████████████████| 181 kB 49.2 MB/s 
     |████████████████████████████████| 144 kB 53.5 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 


In [2]:
#Importin all the used libraries 
import sys
import tensorflow as tf
import numpy as np
from PIL import Image
import pandas as pd
from tensorflow.keras import layers, Model,Input
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image_dataset_from_directory as image_generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
#Path to the directory
Data_Directory = '/content/drive/MyDrive/inaturalist_12K/train'

In [4]:
#Generating data from the directory 
def Data_Generator(Data_Aug):
  if Data_Aug ==False:
    image_Data_generator=ImageDataGenerator(rescale=1./255,validation_split=0.1,preprocessing_function=None,data_format=None)
  else:
    image_Data_generator=ImageDataGenerator(rescale=1./255,validation_split=0.1,featurewise_center=False,
    zca_epsilon=1e-06,
    rotation_range=25,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    preprocessing_function=None,
    data_format='channels_last',
    dtype=None)
  
  #Final Train and Validation data
  Train_Data=image_Data_generator.flow_from_directory(Data_Directory, shuffle = True,seed = 20, subset = 'training')
  Validation_Data=image_Data_generator.flow_from_directory(Data_Directory, shuffle = True,seed = 20, subset = 'validation')

  #You Can select only some percent of data
  #Train_Data = Train_Data.take(int(0.20* len(Train_Data)))
  #Validation_Data = Validation_Data.take(int(0.20*len(Validation_Data)))

  return Train_Data,Validation_Data


In [5]:
#Setting the Sweep Parameters
Sconfig = {'name': 'Test-Sweep', 'method': 'grid'}
parameters_dict = {
                   'FirstLayerFilters': {'values': [64, 32]},
                   'DenseSize': {'values': [128, 64, 32]},
                   'FilterOrg': {'values': [1,0.5,  2]},
                   'BatchNorm': {'values': [True, False]},
                   'DataAug': {'values': [True, False]},
                   'Dropout': {'values': [0.2, 0.0, 0.3]},
                   'KerSize': {'values': [3, 5, 7]}}
#main sweep config
Sconfig['parameters'] = parameters_dict

In [6]:
#Building the model
def Main_Model(Train_Data,Validation_Data,Shape,Activation,PoolSize,First_Layer_Filter,Filter_Org,Convolution_Layers,optimizer,Dense_Size,Kernel_Size,Data_Aug,Dropout,Batch_Normalization,epoch):
  model=tf.keras.Sequential()
  model.add(layers.Input(Shape))
  Filter_Size=[]

  for Layer_Val in range(Convolution_Layers):
    filter_size=(Filter_Org**Layer_Val)*First_Layer_Filter
    model.add(layers.Conv2D(kernel_size=(Kernel_Size,Kernel_Size),filters=filter_size))
    
    if Batch_Normalization:
      model.add(layers.BatchNormalization(axis=-1))
  
  model.add(layers.Activation(Activation))
  model.add(layers.MaxPooling2D(pool_size=(PoolSize,PoolSize),strides=None))

  if Dropout>0:
    model.add(layers.Dropout(Dropout))
  

  model.add(layers.Flatten())
  model.add(layers.Dense(Dense_Size))


  if Batch_Normalization:
    model.add(layers.BatchNormalization(axis=-1))
  model.add(layers.Activation(Activation))

  if Dropout>0:
    model.add(layers.Dropout(Dropout))
  model.add(layers.Dense(10,activation='softmax'))
  return model

In [7]:
def Start_Sweep(config=Sconfig):
  with wandb.init(config=config):
    config=wandb.config
    wandb.run.name=f"Dropout_{config.Dropout}_FilterOrg_{config.FilterOrg}_DenseSize_{config.DenseSize}_BatchNorm_{config.BatchNorm}_FirstLayerFilters_{config.FirstLayerFilters}_KerSize_{config.KerSize}_DataAug_{config.DataAug}"
    
    #Initializing the Parameters through sweep config
    Shape=(256,256,3)
    Activation='relu'
    PoolSize=2
    First_Layer_Filter=config.FirstLayerFilters
    Filter_Org=config.FilterOrg
    Convolution_Layers=5
    optimizer='adam'
    Dense_Size=config.DenseSize
    Kernel_Size=config.KerSize
    Data_Aug=config.DataAug
    Dropout=config.Dropout
    Batch_Normalization=config.BatchNorm
    epoch=50
    Train_Data,Validation_Data=Data_Generator(Data_Aug)

    #Calling Main_Model
    model=Main_Model(Train_Data,Validation_Data,Shape,Activation,PoolSize,First_Layer_Filter,Filter_Org,Convolution_Layers,optimizer,Dense_Size,Kernel_Size,Data_Aug,Dropout,Batch_Normalization,epoch)
    model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics = ["accuracy"])
    model.fit(Train_Data,epochs=epoch,validation_data=Validation_Data,callbacks=[wandb.keras.WandbCallback(),tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4)])
  

In [ ]:
#Making Sweep Functionality
sweep_id = wandb.sweep(Sconfig, project = 'DL_Assignment2_PartA(1)')
wandb.agent(sweep_id, function=Start_Sweep)

In [8]:
#Code Without Wandb
#Uncomment to Execute
"""#Pass the Input Parmeters
Parameters = sys.argv[1:]
Shape=(256,256,3)
Activation=Parameters[1]
PoolSize=int(Parameters[3])
First_Layer_Filter=int(Parameters[5])
Filter_Org=float(Parameters[7])
Convolution_Layers=int(Parameters[9])
optimizer=Parameters[11]
Dense_Size=int(Parameters[13])
Kernel_Size=int(Parameters[15])
Data_Aug=Parameters[17]
Dropout=float(Parameters[19])
Batch_Normalization=Parameters[21]
epoch=int(Parameters[23])

#Generating the Data
Train_Data,Validation_Data=Data_Generator(True)
#Building the Model
model=Main_Model(Train_Data,Validation_Data,Shape,Activation,PoolSize,First_Layer_Filter,Filter_Org,Convolution_Layers,optimizer,Dense_Size,Kernel_Size,Data_Aug,Dropout,Batch_Normalization,epoch)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics = ["accuracy"])
model.fit(Train_Data,epochs=10,validation_data=Validation_Data)


"""

'#Pass the Input Parmeters\nParameters = sys.argv[1:]\nShape=(256,256,3)\nActivation=Parameters[1]\nPoolSize=int(Parameters[3])\nFirst_Layer_Filter=int(Parameters[5])\nFilter_Org=float(Parameters[7])\nConvolution_Layers=int(Parameters[9])\noptimizer=Parameters[11]\nDense_Size=int(Parameters[13])\nKernel_Size=int(Parameters[15])\nData_Aug=Parameters[17]\nDropout=float(Parameters[19])\nBatch_Normalization=Parameters[21]\nepoch=int(Parameters[23])\n\n#Generating the Data\nTrain_Data,Validation_Data=Data_Generator(True)\n#Building the Model\nmodel=Main_Model(Train_Data,Validation_Data,Shape,Activation,PoolSize,First_Layer_Filter,Filter_Org,Convolution_Layers,optimizer,Dense_Size,Kernel_Size,Data_Aug,Dropout,Batch_Normalization,epoch)\nmodel.compile(optimizer=\'adam\',loss=\'categorical_crossentropy\',metrics = ["accuracy"])\nmodel.fit(Train_Data,epochs=10,validation_data=Validation_Data)\n\n\n'